In [1]:
from glob import glob

import email
import smtplib
import imaplib
from getpass import getpass
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

In [2]:
def str2int(message):
    exp = 1
    number = 0
    for i in range(len(message)):
        number += ord(message[i]) * exp
        exp *= 256
    
    return number

def int2str(number):
    message = ''
    while number != 0:
        temp = number % 256
        message += chr(temp)
        number -= temp
        number = number // 256
    
    return message

In [3]:
def encrypt(message, public_key):
    n, e = public_key
    m = message
    L = int(log(n, 256))
    l = int(log(n, 256) / 2)
    parts = []
    while len(m) > L:
        r = rd.randint(l, L)
        aux, m = m[:r], m[r:]
        parts.append(aux)
    else:
        parts.append(m)
        
    encrypted = ''
    for part in parts:       
        part = str2int(part)
        part = pow(part, e, n)
        encrypted += chr(256) + int2str(part)
    
    return encrypted

def decrypt(encrypted, private_key):
    encrypted = encrypted.split(chr(256))
    n, d = private_key
    message = ''
    encrypted.remove('')
    for part in encrypted:
        m = str2int(part)
        m = pow(m, d, n)
        message += int2str(m)
    
    return message

In [4]:
def read_email(username, password, private_key, tag = '[Encrypted]', qtd = 1):
    printed = 0
    crip = False
    server = 'imap.gmail.com'
    mail = imaplib.IMAP4_SSL(server)
    mail.login(username, password)
    mail.select('inbox')
    data = mail.search(None, 'ALL')
    mail_ids = data[1]
    id_list = mail_ids[0].split()   
    first_email_id = int(id_list[0])
    latest_email_id = int(id_list[-1])
    for i in range(latest_email_id, first_email_id, -1):
        if printed == qtd:
            break

        data = mail.fetch(str(i), '(RFC822)')
        for response_part in data:
            arr = response_part[0]
            if isinstance(arr, tuple):
                msg = email.message_from_string(str(arr[1], 'utf-8'))
                email_subject = msg['subject']
                email_from = msg['from']
                if tag not in email_subject:
                    break
                else:
                    printed += 1
                    crip = True

                print('From: ' + email_from)
                print('Subject: ' + email_subject)
                print()
                for part in msg.walk():
                    content_type = part.get_content_type()
                    content_disposition = str(part.get('Content-Disposition'))
                    try:
                        body = part.get_payload(decode = True).decode()
                    except:
                        pass

                    if content_type == 'text/plain':
                        body = decrypt(body, private_key)
                        print(body)
        
        if printed != qtd and crip:
            print()
            crip = False

In [5]:
if 'public.csv' not in glob('*.csv'):
    public_key, private_key = generate_keys(p, q)
    with open('public.csv', 'w') as file:
        file.write(str(public_key[0]) + '\n')
        file.write(str(public_key[1]) + '\n')
        
    with open('private.csv', 'w') as file:
        file.write(str(private_key[0]) + '\n')
        file.write(str(private_key[1]) + '\n')   
else:
    file = open('public.csv')
    public_key = file.readlines()
    file.close()
    public_key = tuple([int(i) for i in public_key])
    
    file = open('private.csv')
    private_key = file.readlines()
    file.close()
    private_key = tuple([int(i) for i in private_key])

In [6]:
username = input('Username: ')
password = getpass(prompt = 'Password: ')

Username: luca.escopelli@gmail.com
Password: ········


In [7]:
read_email(username, password, private_key)

From: igorpmichels@gmail.com
Subject: [Encrypted] Teste

Não tinha medo o tal João de Santo Cristo
Era o que todos diziam quando ele se perdeu
Deixou pra trás todo o marasmo da fazenda
Só pra sentir no seu sangue o ódio que Jesus lhe deu
Quando criança só pensava em ser bandido
Ainda mais quando com um tiro de soldado o pai morreu
Era o terror da sertania onde morava
E na escola até o professor com ele aprendeu
Ia pra igreja só pra roubar o dinheiro
Que as velhinhas colocavam na caixinha do altar
Sentia mesmo que era mesmo diferente
Sentia que aquilo ali não era o seu lugar
Ele queria sair para ver o mar
E as coisas que ele via na televisão
Juntou dinheiro para poder viajar
De escolha própria, escolheu a solidão
Comia todas as menininhas da cidade
De tanto brincar de médico, aos doze era professor
Aos quinze, foi mandado pro o reformatório
Onde aumentou seu ódio diante de tanto terror
Não entendia como a vida funcionava
Discriminação por causa da sua classe e sua cor
Ficou cansado de t